## Packages Installation

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

In [1]:
import warnings

import pandas as pd
from holisticai.bias.metrics import clustering_bias_metrics
from holisticai.datasets import load_dataset
from holisticai.pipeline import Pipeline
from sklearn.cluster import KMeans

warnings.filterwarnings("ignore")

## Data Loading

In [2]:
dataset = load_dataset('compass',protected_attribute="race")
train_test = dataset.train_test_split(test_size=0.2, random_state=42)

train = train_test['train']
test = train_test['test']

dataset

{"dtype":"Dataset","attributes":{"Instances":16644,"Features":["X , y , p_attrs , group_a , group_b"]},"metadata":"race: {'group_a': 'Causasian', 'group_b': 'Non-Caucasian'}"}

In [3]:
from holisticai.bias.mitigation import FairletClusteringPreprocessing

# set the model
model = KMeans(n_clusters = 3, random_state=42)
model.fit(train['X'])
y_pred = model.predict(train['X'])
centroids = model.cluster_centers_

# get the metrics
metrics = clustering_bias_metrics(train['group_a'], train['group_b'], y_pred, data = train['X'], centroids = centroids, metric_type = 'equal_outcome')
metrics

,Value,Reference
Metric,,
Cluster Balance,0.319848,1
Minimum Cluster Ratio,1.064137,1
Cluster Distribution Total Variation,0.048022,0
Cluster Distribution KL Div,0.009896,0
Social Fairness Ratio,1.042285,1
Silhouette Difference,0.004703,0


# 1. Fairlet 

#### Traditional Implementation

In [4]:
from holisticai.bias.mitigation import FairletClusteringPreprocessing

# set the model
model = KMeans(n_clusters = 3, random_state=42)

# set the mitigator and fit the model
mitigator = FairletClusteringPreprocessing(seed=42)
Xpre = mitigator.fit_transform(train['X'], train['group_a'], train['group_b'])
model.fit(Xpre)

# predict the clusters and get the centroids
y_pred = model.predict(Xpre)
centroids = model.cluster_centers_

# get the metrics
metrics = clustering_bias_metrics(train['group_a'], train['group_b'], y_pred, data = train['X'], centroids = centroids, metric_type = 'equal_outcome')
metrics

,Value,Reference
Metric,,
Cluster Balance,0.922569,1
Minimum Cluster Ratio,1.000000,1
Cluster Distribution Total Variation,0.013492,0
Cluster Distribution KL Div,0.000419,0
Social Fairness Ratio,1.041066,1
Silhouette Difference,0.132821,0


# Pipeline Implementation

In [5]:
mitigator = FairletClusteringPreprocessing(seed=42)

# set the pipeline
pipeline = Pipeline(steps=[('bm_preprocessing', mitigator), ('model', model)])
pipeline.fit(train['X'], bm__group_a = train['group_a'], bm__group_b = train['group_b'])

# predict the clusters and get the centroids
y_pred = pipeline.predict(train['X'])
centroids = model.cluster_centers_

# compute the bias metrics
metrics_pipeline = clustering_bias_metrics(train['group_a'], train['group_b'], y_pred, data = train['X'], centroids = centroids, metric_type = 'equal_outcome')
metrics_pipeline

,Value,Reference
Metric,,
Cluster Balance,0.932819,1
Minimum Cluster Ratio,1.022222,1
Cluster Distribution Total Variation,0.011651,0
Cluster Distribution KL Div,0.000312,0
Social Fairness Ratio,1.041066,1
Silhouette Difference,0.131541,0


### Comparison

In [6]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                           Value     Value Reference
Metric                                                              
Cluster Balance                         0.922569  0.932819         1
Minimum Cluster Ratio                   1.000000  1.022222         1
Cluster Distribution Total Variation    0.013492  0.011651         0
Cluster Distribution KL Div             0.000419  0.000312         0
Social Fairness Ratio                   1.041066  1.041066         1
Silhouette Difference                   0.132821  0.131541         0